In [1]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
#we want to create a neural network, so we need to import the keras library
from keras.models import Sequential
import tensorflow as tf
from sklearn.ensemble import IsolationForest

In [2]:
csv_file = "2500-40000K.csv" 
df = pd.read_csv(csv_file, sep=",")
# individual photometry filters
bp_rp = df['bp_rp']
bp_g = df['bp_g']
g_rp = df['g_rp']

# Effective temperature of star
temperature = df['teff_gspphot']

color_filters_df = pd.DataFrame()
color_filters_df['bp_rp'] = bp_rp
color_filters_df['bp_g'] = bp_g
color_filters_df['g_rp'] = g_rp

# Effective temperature of star
color_filters_df['temperature'] = df['teff_gspphot']

In [3]:
#the input of the model is the color indices, and the output is the temperature.
#lets perform this with a neural network
#we need to split the data into training and testing data
#we will use 80% of the data for training and 20% for testing
#we will also shuffle the data to avoid any bias

X = color_filters_df[['bp_rp', 'bp_g', 'g_rp']].values
y = color_filters_df['temperature'].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)


In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)), #change input shape based on how many u using
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  #1 neuron output
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [5]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
6346/6346 [==============================] - 22s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/20
6346/6346 [==============================] - 20s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/20
6346/6346 [==============================] - 20s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/20
6346/6346 [==============================] - 26s 4ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/20
6346/6346 [==============================] - 22s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/20
6346/6346 [==============================] - 20s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/20
 595/6346 [=>............................] - ETA: 16s - loss: nan - mae: nan

In [8]:
predictions = model.predict(X_test)

#assign max to a really low number
max = -10000000
#assign min to a really high number
min = 10000000

avg = 0

for i in range(len(X_test)):
    # print("Input:", X_test[i])
    #if actual output is greater than 250000, then pritn it
    if predictions[i][0] > 250000: # HAD y_test[i] == 4353
        print("Input:", X_train[i])
        print("Expected Output:", y_test[i])
        print("Actual Output:", predictions[i][0])  # predictions[i] is a 2D array, so we access the value with [0]
        print()
    #update max and min accordingly, based on difference in expected and actual output
    if abs(y_test[i] - predictions[i][0]) > max:
        max = abs(y_test[i] - predictions[i][0])
    if abs(y_test[i] - predictions[i][0]) < min:
        min = abs(y_test[i] - predictions[i][0])
    avg += abs(y_test[i] - predictions[i][0])
avg = avg / len(X_test)
# print(min, max, avg)

1983/1983 [==============================] - 1s 242us/step


In [9]:
r2 = r2_score(y_test, predictions)
print("R2 Score:", r2)

ValueError: Input contains NaN.

In [ ]:
plt.scatter(y_test, predictions, c='b', marker='o', label='Predictions')

# Add labels and a 1:1 line for reference
plt.xlabel('Actual')
plt.ylabel('metallicity outliers: Predictions')
plt.plot(color='red', linestyle='--', label='1:1 Line')

# Add a legend
plt.legend()

# Show the plot
plt.title("Scatter Plot of Actual vs. Predictions")
plt.grid(True)
plt.show()